# The Generated WebGME code

In [1]:
# deep learning -----------------------------------------------------
# to build own CNN from scratch
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, BatchNormalization, Activation, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import pandas as pd
import numpy as np
# preprocess initial data to feed itto CNN
train = pd.read_csv('../Datasets/MNIST_dataset/train.csv', engine='c', sep=',')
test = pd.read_csv('../Datasets/MNIST_dataset/test.csv', engine='c', sep=',')

In [2]:
batch_size = 128
num_classes = 10
epochs = 5
# input image dimensions
img_rows, img_cols = 28, 28

train = train.values

train_x = train[:,1:].reshape(train.shape[0], img_rows, img_cols, 1).astype('float32') / 255.
train_y = to_categorical(train[:, 0], num_classes) # 3 -> [0,0,0,1,0,0,0,0,0,0]


test = test.values
test_x = test.reshape(test.shape[0], img_rows, img_cols, 1).astype('float32') / 255.
test_y = to_categorical(test[:, 0], num_classes) # 3 -> [0,0,0,1,0,0,0,0,0,0]

print('x_train shape:', train_x.shape)
print(train_x.shape[0], 'train samples')
print(test_x.shape[0], 'test samples')

In [3]:
%%time

from keras.callbacks import TensorBoard

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=True)

# let's build simple convolutional network from scratch
model = Sequential()

# 1st set of layers
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=(img_rows, img_cols, 1)))
# Normalize the activations of the previous layer at each batch - apply a transformation 
# that maintains the mean activation close to 0 and the activation standard deviation close to 1.
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25)) # regularization

# 2nd set of layers
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# fully-connected layers, flatten previous output and connect n x m
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 10 digits -> 10 classes
model.add(Dense(num_classes))
model.add(Activation('softmax')) # probabilistic output

# compile model
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax', 
              metrics=['accuracy']
             )

# let's explore model structure
#model.summary()
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(test_x, test_y),
          callbacks=[tensorboard])

In [4]:
# The predict_classes function outputs the highest probability class
# according to the trained classifier for each input example.
predicted_classes = model.predict_classes(test_x)

import matplotlib.pyplot as plt
%matplotlib inline
fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
for i, pred in enumerate(predicted_classes[5000:5050]):
    fig.add_subplot(5,10,i+1)
    plt.subplots_adjust(hspace=.1)
    plt.imshow(test_x[pred].reshape(28,28), cmap='gray', interpolation='none')
    #plt.title("Predicted {}, Class {}".format(predicted_classes[correct], test_y[correct]))
    plt.title("Predicted {}".format(predicted_classes[pred]))